# Multi-View 3D Reconstruction using COLMAP

This notebook demonstrates the process of 3D reconstruction from multiple images using COLMAP, a general-purpose Structure-from-Motion (SfM) and Multi-View Stereo (MVS) pipeline.

## Step 1: Download COLMAP Source Code
First, we'll download the COLMAP source code from Google Drive.

In [1]:
import gdown

url = 'https://drive.google.com/file/d/1-Q_lZ7wW7gCWTWV-FWTGEjOZPEnPG7gE/view?usp=drive_link'
output_path = 'colmap.zip'
gdown.download(url, output_path, quiet=False,fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1-Q_lZ7wW7gCWTWV-FWTGEjOZPEnPG7gE
From (redirected): https://drive.google.com/uc?id=1-Q_lZ7wW7gCWTWV-FWTGEjOZPEnPG7gE&confirm=t&uuid=34f32261-2089-48dd-8489-e88ec935f2a7
To: /content/colmap.zip
100%|██████████| 119M/119M [00:03<00:00, 35.0MB/s]


'colmap.zip'

## Step 2: Extract the COLMAP Source
Extract the downloaded zip file to the root directory.

In [2]:
!unzip /content/colmap.zip -d /

Archive:  /content/colmap.zip
   creating: /content/colmap/
  inflating: /content/colmap/.clang-tidy  
  inflating: /content/colmap/vcpkg.json  
   creating: /content/colmap/cmake/
  inflating: /content/colmap/cmake/FindFreeImage.cmake  
  inflating: /content/colmap/cmake/FindDependencies.cmake  
  inflating: /content/colmap/cmake/FindGlog.cmake  
  inflating: /content/colmap/cmake/colmap-config-version.cmake.in  
  inflating: /content/colmap/cmake/CMakeUninstall.cmake.in  
  inflating: /content/colmap/cmake/FindCryptoPP.cmake  
  inflating: /content/colmap/cmake/FindLZ4.cmake  
  inflating: /content/colmap/cmake/CMakeHelper.cmake  
  inflating: /content/colmap/cmake/FindMetis.cmake  
  inflating: /content/colmap/cmake/GenerateVersionDefinitions.cmake  
  inflating: /content/colmap/cmake/FindFLANN.cmake  
  inflating: /content/colmap/cmake/FindGlew.cmake  
  inflating: /content/colmap/cmake/colmap-config.cmake.in  
   creating: /content/colmap/docker/
  inflating: /content/colmap/docke

## Step 3: Install Dependencies
COLMAP requires several libraries and dependencies. This step installs them using apt-get.

In [3]:
!sudo apt-get install \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libgmock-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev set to manually installed.
libboost-graph-dev is already the newest version (1.74.0.3ubuntu7).
libboost-graph-dev set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.12).
libsqlite3-dev is already the newest version (3.37.2-2ubuntu0.3).
libsqlite3-dev set to manually installed.
The following additional packages will be installed:
  googletest libamd2 libbtf1 libcamd2 libccolamd2 libceres2 libcholmod3
  libcolamd2 libcxsparse3 libegl-dev libevdev2 libflann1.9 libfreeimage3
  libgflags-dev libgflags2.2 libgl-dev libglu1-mesa

## Step 4: Install COLMAP
Navigate to the build directory and install COLMAP using Ninja.

In [4]:
%cd /content/colmap/build
! ninja install

/content/colmap/build
[0/1] Install the project...
-- Install configuration: "Release"
-- Installing: /usr/local/share/applications/COLMAP.desktop
-- Installing: /usr/local/lib/libcolmap_controllers.a
-- Installing: /usr/local/lib/libcolmap_estimators.a
-- Installing: /usr/local/lib/libcolmap_exe.a
-- Installing: /usr/local/lib/libcolmap_feature_types.a
-- Installing: /usr/local/lib/libcolmap_feature.a
-- Installing: /usr/local/lib/libcolmap_geometry.a
-- Installing: /usr/local/lib/libcolmap_image.a
-- Installing: /usr/local/lib/libcolmap_math.a
-- Installing: /usr/local/lib/libcolmap_mvs.a
-- Installing: /usr/local/lib/libcolmap_optim.a
-- Installing: /usr/local/lib/libcolmap_retrieval.a
-- Installing: /usr/local/lib/libcolmap_scene.a
-- Installing: /usr/local/lib/libcolmap_sensor.a
-- Installing: /usr/local/lib/libcolmap_sfm.a
-- Installing: /usr/local/lib/libcolmap_util.a
-- Installing: /usr/local/lib/libcolmap_poisson_recon.a
-- Installing: /usr/local/lib/libcolmap_vlfeat.a
-- Inst

## Step 5: Clone the Dataset Repository
Clone the repository containing the dataset we'll use for reconstruction.

In [5]:
%cd /content/
! git clone https://github.com/VarunBurde/Prague_ml.git

/content
Cloning into 'Prague_ml'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 232 (delta 10), reused 15 (delta 6), pack-reused 213 (from 4)
Receiving objects: 100% (232/232), 235.09 MiB | 27.25 MiB/s, done.
Resolving deltas: 100% (46/46), done.


## Step 6: Prepare the Dataset
Set up the reconstruction directory and create a COLMAP database. COLMAP uses this SQLite database to store feature information about each image.

In [30]:
import os
import shutil

dataset_path = "/content/Prague_ml/dataset/ricaip"
output_dir = "/content/reconstruction"

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
shutil.copytree(dataset_path, output_dir)

database_path = os.path.join(output_dir, "database.db")
image_path = os.path.join(output_dir, "images")
!colmap database_creator --database_path {database_path}


## Step 7: Extract Features
Extract SIFT features from each image. We specify OPENCV camera model and indicate all images are from a single camera.

In [31]:
!colmap feature_extractor \
    --database_path {database_path} \
    --image_path {image_path} \
    --ImageReader.camera_model OPENCV \
    --ImageReader.single_camera 1

I0425 17:45:03.183815  7522 misc.cc:44] 
Feature extraction
I0425 17:45:03.184568  7525 sift.cc:721] Creating SIFT GPU feature extractor
I0425 17:45:03.811848  7526 feature_extraction.cc:258] Processed file [1/20]
I0425 17:45:03.811892  7526 feature_extraction.cc:261]   Name:            image_1745412808.png
I0425 17:45:03.811899  7526 feature_extraction.cc:270]   Dimensions:      2448 x 2048
I0425 17:45:03.811908  7526 feature_extraction.cc:273]   Camera:          #1 - OPENCV
I0425 17:45:03.811915  7526 feature_extraction.cc:276]   Focal Length:    2937.60px
I0425 17:45:03.811932  7526 feature_extraction.cc:280]   Features:        8599
I0425 17:45:04.009855  7526 feature_extraction.cc:258] Processed file [2/20]
I0425 17:45:04.009896  7526 feature_extraction.cc:261]   Name:            image_1745412812.png
I0425 17:45:04.009904  7526 feature_extraction.cc:270]   Dimensions:      2448 x 2048
I0425 17:45:04.009912  7526 feature_extraction.cc:273]   Camera:          #1 - OPENCV
I0425 17:45:

## Step 8: Match Features
Match features between images using exhaustive matching. This compares every image against every other image to find correspondences.

In [32]:
!colmap exhaustive_matcher \
    --database_path {database_path}


I0425 17:45:10.738957  7561 misc.cc:44] 
Feature matching
I0425 17:45:10.739564  7562 sift.cc:1426] Creating SIFT GPU feature matcher
I0425 17:45:10.842010  7561 pairing.cc:168] Generating exhaustive image pairs...
I0425 17:45:10.843364  7561 pairing.cc:201] Matching block [1/1, 1/1]
I0425 17:45:13.071894  7561 feature_matching.cc:46] in 2.229s
I0425 17:45:13.081876  7561 timer.cc:91] Elapsed time: 0.039 [minutes]


## Step 9: Sparse Reconstruction
Perform the sparse reconstruction (Structure-from-Motion) to estimate camera poses and a sparse 3D point cloud.

In [ ]:
sparse_folder_path = os.path.join(output_dir, "sparse")
if os.path.exists(sparse_folder_path):
    shutil.rmtree(sparse_folder_path)

os.makedirs(sparse_folder_path)

!colmap mapper \
    --database_path {database_path}  \
    --image_path {image_path} \
    --output_path {sparse_folder_path} \
    --Mapper.init_min_tri_angle 2 \
    --Mapper.abs_pose_min_num_inliers 10 \
    --Mapper.ba_refine_focal_length 1 \
    --Mapper.ba_refine_principal_point 1


I0425 17:48:39.778082  8505 incremental_pipeline.cc:251] Loading database
I0425 17:48:39.779633  8505 database_cache.cc:66] Loading cameras...
I0425 17:48:39.779675  8505 database_cache.cc:76]  1 in 0.000s
I0425 17:48:39.779687  8505 database_cache.cc:84] Loading matches...
I0425 17:48:39.785910  8505 database_cache.cc:89]  190 in 0.006s
I0425 17:48:39.785935  8505 database_cache.cc:105] Loading images...
I0425 17:48:39.800076  8505 database_cache.cc:153]  20 in 0.014s (connected 20)
I0425 17:48:39.800103  8505 database_cache.cc:164] Loading pose priors...
I0425 17:48:39.800228  8505 database_cache.cc:175]  0 in 0.000s
I0425 17:48:39.800240  8505 database_cache.cc:184] Building correspondence graph...
I0425 17:48:39.894946  8505 database_cache.cc:210]  in 0.095s (ignored 0)
I0425 17:48:39.895040  8505 timer.cc:91] Elapsed time: 0.002 [minutes]
I0425 17:48:39.896627  8505 incremental_pipeline.cc:297] Finding good initial image pair
I0425 17:48:39.945003  8505 incremental_pipeline.cc:321

## Step 10: Image Undistortion
Undistort the images using the camera parameters estimated during sparse reconstruction. This prepares the images for dense reconstruction.

In [ ]:
dense_folder_path = os.path.join(output_dir, "dense")
if os.path.exists(dense_folder_path):
    shutil.rmtree(dense_folder_path)

os.makedirs(dense_folder_path)

!colmap image_undistorter \
    --image_path {image_path} \
    --input_path {os.path.join(output_dir, "sparse/0")} \
    --output_path {dense_folder_path} \
    --output_type COLMAP \
    --max_image_size 2000

## Step 11: Dense Stereo Matching
Perform multi-view stereo to generate depth maps for each image using PatchMatch stereo with geometric consistency.

In [ ]:
!colmap patch_match_stereo \
    --workspace_path {dense_folder_path} \
    --workspace_format COLMAP \
    --PatchMatchStereo.geom_consistency true


## Step 12: Stereo Fusion
Fuse the depth maps to create a dense point cloud (PLY file).

In [ ]:
fused_ply_path = os.path.join(dense_folder_path, "fused.ply")

!colmap stereo_fusion \
    --workspace_path {dense_folder_path} \
    --workspace_format COLMAP \
    --input_type geometric \
    --output_path {fused_ply_path}

## Step 13: Surface Reconstruction
Create a 3D mesh from the dense point cloud using Poisson surface reconstruction.

In [ ]:
output_ply_path = os.path.join(dense_folder_path, "meshed-poisson.ply")

!colmap poisson_mesher \
    --input_path {fused_ply_path} \
    --output_path {output_ply_path}

## Results

The reconstruction process has created several outputs:

1. **Sparse Point Cloud**: Located in `/content/reconstruction/sparse/0`
2. **Dense Point Cloud**: Located at `/content/reconstruction/dense/fused.ply`
3. **3D Mesh**: Located at `/content/reconstruction/dense/meshed-poisson.ply`

You can download these files to visualize them in software like MeshLab or CloudCompare.